In [3]:
import json

# from sheets_api import GoogleSheetsManager
from src import sheets_api
import gspread as gs
import awswrangler as wr
import pandas as pd

In [46]:
df = pd.DataFrame(
    columns=["report_id", "site_id", "site_name", "lat", "lng", "catchment_type", "top_brands", 'pois', "projects",
             'apartments', 'median_price', 'household_distribution', 'competition', "population", "affluence",
             "avg_cost_for_two", "revenue_score", 'high_streets', 'shopping_malls', 'poi_counts', 'projects_counts',
             'created_at'])
df

,report_id,site_id,site_name,lat,lng,catchment_type,top_brands,pois,projects,apartments,...,household_distribution,competition,population,affluence,avg_cost_for_two,revenue_score,high_streets,shopping_malls,poi_counts,projects_counts


In [47]:
wr.s3.to_parquet(df, path="s3://tuzomldev/site_reports/", table="ind_site_reports_data_v2", database='datasets_prep',
                 dataset=True, mode='append',
                 dtype={'report_id': 'string', 'site_id': 'string', 'site_name': 'string', 'lat': 'double',
                        'lng': 'double', 'catchment_type': 'string', 'top_brands': 'string', 'pois': 'string',
                        'projects': 'string', 'apartments': 'string', 'median_price': 'string',
                        'household_distribution': 'string', 'competition': 'string', 'population': 'string',
                        'affluence': 'double', 'avg_cost_for_two': 'double', 'revenue_score': 'double',
                        'high_streets': 'string', 'shopping_malls': 'string', 'poi_counts': 'string',
                        'projects_counts': 'string', 'created_at': 'bigint'})

{'paths': ['s3://tuzomldev/site_reports/c93c4a30641a4279975a9d14b0a8e68e.snappy.parquet'],
 'partitions_values': {}}

In [2]:
manager = sheets_api.GoogleSheetsManager()

In [25]:
sh = manager.get_sheet_by_id("1COD7ZJpmc1NxKlw8QXyfgglCXNmMc99Wrx2vjbLlsJI")

In [7]:
sh.duplicate_sheet()

<Spreadsheet 'Copy of Site Report - Orion Mall' id:1COD7ZJpmc1NxKlw8QXyfgglCXNmMc99Wrx2vjbLlsJI>

In [12]:
new_sh = manager.client.copy(file_id='1COD7ZJpmc1NxKlw8QXyfgglCXNmMc99Wrx2vjbLlsJI', title='Site Report - Test',
                             folder_id='1ILAdxTEs8wxWkH_7vLRm9mk04CNzd9sH')

In [13]:
new_sh

<Spreadsheet 'Site Report - Test' id:12CbqlizKSN7r66hpytciM_s6McKjsbiJeuNejMaDjpU>

In [38]:
sheets = sh.fetch_sheet_metadata()['sheets']

In [39]:
sheets = {sheet['properties']['title']: sheet['properties']['index'] for sheet in sheets}

In [40]:
sheets

{'Site Report': 0,
 'Sheet5': 1,
 'projects': 2,
 'pois_ranked': 3,
 'Copy of pois_ranked': 4,
 'competition': 5,
 'high_streets': 6}

In [42]:
import requests
import pandas as pd

In [6]:
ws = sh.get_worksheet(0)

In [7]:
ws.url

'https://docs.google.com/spreadsheets/d/12CbqlizKSN7r66hpytciM_s6McKjsbiJeuNejMaDjpU#gid=0'

In [40]:
# with open("dashboard.json",'w') as f:
#     json.dump(up,f,indent=4)

In [15]:
ranges_to_clear = []
with open("dashboard_copy.json") as f:
    data = json.load(f)
for i in data:
    ranges_to_clear.append(i['range'])

In [16]:
ranges_to_clear += ["E85:K92", 'E107:I110', 'E62:K66', 'E121:K124', 'N18:O35', 'E74:H79', 'E99:H103', 'F5', 'F17']

In [17]:
ws.batch_clear(ranges_to_clear)

{'spreadsheetId': '12CbqlizKSN7r66hpytciM_s6McKjsbiJeuNejMaDjpU',
 'clearedRanges': ["'Site Report'!F9",
  "'Site Report'!G9",
  "'Site Report'!E23",
  "'Site Report'!F10",
  "'Site Report'!G10",
  "'Site Report'!G23",
  "'Site Report'!H26",
  "'Site Report'!H27",
  "'Site Report'!H28",
  "'Site Report'!H29",
  "'Site Report'!H30",
  "'Site Report'!F11",
  "'Site Report'!L26",
  "'Site Report'!G11",
  "'Site Report'!F12",
  "'Site Report'!G12",
  "'Site Report'!J23",
  "'Site Report'!J9",
  "'Site Report'!K9",
  "'Site Report'!J12",
  "'Site Report'!K12",
  "'Site Report'!L23",
  "'Site Report'!G134",
  "'Site Report'!I134",
  "'Site Report'!G140",
  "'Site Report'!G143",
  "'Site Report'!E134",
  "'Site Report'!G118",
  "'Site Report'!E118",
  "'Site Report'!J118",
  "'Site Report'!E85:K92",
  "'Site Report'!E107:I110",
  "'Site Report'!E62:K66",
  "'Site Report'!E121:K124",
  "'Site Report'!N18:O35",
  "'Site Report'!E74:H79",
  "'Site Report'!E99:H103",
  "'Site Report'!F5",
  "'Sit

In [162]:
def get_dictionary_value(d, keys):
    keys = keys.split(".")
    for i in keys:
        d = d.get(i)
    return d

In [43]:
eps = ["cft", "population", "companies", "education", "projects", "affluence", "income", "competition",
       "category_count", "property_price", "medical", "high_street", ]
all_data = {"data": {}}
for ep in eps:
    url = f"http://127.0.0.1:8000/site_report/1/{ep}"
    req = requests.get(url)
    resp = req.json()
    data = resp['data']
    all_data['data'][ep] = data


In [180]:
with open("dashboard_copy.json") as f:
    data = json.load(f)
for i in data:
    try:
        i['values'] = [[get_dictionary_value(all_data, i['values'][0][0])]]
    except Exception as e:
        print(e, i['values'][0][0])

In [181]:
ws.batch_update(data)

{'spreadsheetId': '12CbqlizKSN7r66hpytciM_s6McKjsbiJeuNejMaDjpU',
 'totalUpdatedRows': 12,
 'totalUpdatedColumns': 8,
 'totalUpdatedCells': 27,
 'totalUpdatedSheets': 1,
 'responses': [{'spreadsheetId': '12CbqlizKSN7r66hpytciM_s6McKjsbiJeuNejMaDjpU',
   'updatedRange': "'Site Report'!F9",
   'updatedRows': 1,
   'updatedColumns': 1,
   'updatedCells': 1},
  {'spreadsheetId': '12CbqlizKSN7r66hpytciM_s6McKjsbiJeuNejMaDjpU',
   'updatedRange': "'Site Report'!G9",
   'updatedRows': 1,
   'updatedColumns': 1,
   'updatedCells': 1},
  {'spreadsheetId': '12CbqlizKSN7r66hpytciM_s6McKjsbiJeuNejMaDjpU',
   'updatedRange': "'Site Report'!E23",
   'updatedRows': 1,
   'updatedColumns': 1,
   'updatedCells': 1},
  {'spreadsheetId': '12CbqlizKSN7r66hpytciM_s6McKjsbiJeuNejMaDjpU',
   'updatedRange': "'Site Report'!F10",
   'updatedRows': 1,
   'updatedColumns': 1,
   'updatedCells': 1},
  {'spreadsheetId': '12CbqlizKSN7r66hpytciM_s6McKjsbiJeuNejMaDjpU',
   'updatedRange': "'Site Report'!G10",
   'upd

In [47]:
all_data['data']['category_count']['15m_catchment']['top_pois']

[{'category': 'clinic',
  'count': 363,
  'top_pois': [{'id': 'ChIJHZok4tc9rjsRYVjWBCjdKs4',
    'name': 'AARTHI SCANS & LABS | RAJAJINAGAR | DIAGNOSTIC CENTER',
    'lat': 12.981183052062988,
    'lng': 77.54934692382812,
    'source': 'gmaps_v2',
    'category': 'clinic',
    'number_of_votes': 3567,
    'reviews_per_day': 2.5627882659481243,
    'distance': 3.35},
   {'id': 'ChIJcU-7k1Y9rjsRmPp0glrR8S8',
    'name': 'Lenskart.com at Rajaji Nagar, 19th Main Road, Bengaluru',
    'lat': 13.000310897827148,
    'lng': 77.55036926269531,
    'source': 'gmaps_v2',
    'category': 'clinic',
    'number_of_votes': 179,
    'reviews_per_day': 1.200839693305095,
    'distance': 1.28},
   {'id': 'ChIJkRpDzoI9rjsRxOMETabtF-A',
    'name': 'SkinSerene Clinic',
    'lat': 12.986992835998535,
    'lng': 77.5421371459961,
    'source': 'gmaps_v2',
    'category': 'clinic',
    'number_of_votes': 73,
    'reviews_per_day': 0.8317527191991901,
    'distance': 2.99},
   {'id': 'ChIJY3xQT2I9rjsRLN8EO-

0     [{'id': 'ChIJHZok4tc9rjsRYVjWBCjdKs4', 'name':...
1     [{'id': 'ChIJzZWOWNsXrjsRLHZGGgj8Eg0', 'name':...
2     [{'id': 'ChIJlb8QkIwXrjsRGUxSwTzIMhQ', 'name':...
3     [{'id': 'ChIJpRY7UWw9rjsRkzbqz8VlSgQ', 'name':...
4     [{'id': 'ChIJCUy6J5AXrjsRI8wWfSwJQNs', 'name':...
5     [{'id': 'ChIJ5b7pxwMWrjsRvAoPkDynRBE', 'name':...
6     [{'id': 'ChIJbyQyMQIXrjsRQfl6H8eQ5Wg', 'name':...
7     [{'id': '60432', 'name': 'Al-Bek', 'lat': 12.9...
8     [{'id': 'ChIJT86eaNc8rjsRqJf0vL_G1DY', 'name':...
9     [{'id': 'ChIJRfEI2DoWrjsRknzICVhGETE', 'name':...
10    [{'id': 'ChIJbQb8IPs9rjsR4j-RbzJFroA', 'name':...
11    [{'id': 'ChIJUx1_EOEXrjsR7GanFYbRTPc', 'name':...
Name: top_pois, dtype: object

,id,name,lat,lng,source,category,number_of_votes,reviews_per_day,distance
1064,ChIJlb8QkIwXrjsRGUxSwTzIMhQ,Lulu Mall Bengaluru,12.981745,77.563789,gmaps_v2,coffee_shop,28438,360.794739,3.39
2765,ChIJbQb8IPs9rjsR4j-RbzJFroA,GT World Mall,12.973810,77.551590,gmaps_v2,shopping_mall,45514,33.927081,4.13
1995,60432,Al-Bek,12.977953,77.553110,zomato,restaurant,71200,23.570983,3.66
2766,ChIJj9VQyNE9rjsRsVrZQsVMFmM,Vaishnavi Sapphire Centre,13.024007,77.548920,gmaps_v2,shopping_mall,30888,19.247213,1.57
1996,18703504,Oottupura,13.033714,77.564107,zomato,restaurant,12700,6.922251,2.71
...,...,...,...,...,...,...,...,...,...
1063,ChIJvRyWtk89rjsR1S9ykaF6PWk,EdQart Experience Store -Rajajinagar,12.999279,77.550812,gmaps_v2,clothing_store,28,0.000000,1.37
1401,ChIJVcadQj4XrjsRU_gNE0o2Fzk,THE HEAVEN CAFFE,13.004108,77.569420,gmaps_v2,coffee_shop,58,0.000000,1.77
1400,ChIJ4UQn6QsXrjsRzvVynZTkyM4,Bhukkad Punjabi cuisine,13.029139,77.567421,gmaps_v2,coffee_shop,24,0.000000,2.43
2764,ChIJDzLCbRI9rjsR8ekg35hA4cI,Metro Motors,13.020294,77.537903,gmaps_v2,shoe_store,47,0.000000,2.09


In [122]:
def update_category_count():
    buff = pd.DataFrame(all_data['data']['category_count']['500_buffer'])[['category', 'count']]
    cat = pd.DataFrame(all_data['data']['category_count']['15m_catchment'])[['category', 'count']]
    cols = {"category": "Category", "count": "Count", "Percentage": "Percentage", "Top Brands": "Top Brands"}
    for comp_df in [buff, cat]:
        for k, v in cols.items():
            if k not in comp_df.columns.tolist():
                comp_df[k] = None
        comp_df.rename(columns=cols, inplace=True)
    df = pd.merge(buff, cat, how='outer', on=['Category'])
    df.columns = [col.replace('_x', '').replace("_y", '') for col in df.columns]
    df = df.replace({pd.NA: None}).fillna({'Count': 0})
    ws.batch_update([{"range": "E85:K92", "values": df.values.tolist()}])


In [189]:
def update_high_streets():
    df = pd.DataFrame(all_data['data']['high_street']['15m_catchment'])[['cluster_name', 'distance', 'area', ]]
    cols = {"cluster_name": "High Streets", "Avg Daily visits": "Avg Daily visits",
            "% of Catchment visitors": "% of Catchment visitors", "distance": "Distance from Site",
            'area': "Total Area"}
    for k, v in cols.items():
        if k not in df.columns.tolist():
            df[k] = None
    df.rename(columns=cols, inplace=True)
    df = df[cols.values()]
    df['Total Area'] = df['Total Area'].apply(lambda x: f"{x / 10 ** 6:.2f} sqkm")
    df['Distance from Site'] = df['Distance from Site'].apply(lambda x: f"{x} km")
    ws.batch_update([{"range": "E107:I110", "values": df.head(4).values.tolist()}])


In [187]:
def update_competition():
    comp_df = pd.DataFrame(all_data['data']['competition']['15m_catchment']['pois'])
    cols = {"name": "Brand", "type": "Type", "distance": "Distance from Location",
            "Avg Daily visits": "Avg Daily visits",
            "Performance": "Performance", "number_of_votes": "# of Google Review", "ratings": "Google ratings"}
    for k, v in cols.items():
        if k not in comp_df.columns.tolist():
            comp_df[k] = None
    comp_df = comp_df.rename(columns=cols)
    comp_df = comp_df[list(cols.values())]
    ws.batch_update([{"range": "E62:K66", "values": comp_df.head(5).values.tolist()}])


In [184]:
def update_projects():
    comp_df = pd.DataFrame(all_data['data']['projects']['15m_catchment']['projects'])
    cols = {"name": "Apartments", "Site Visitor count": "Site Visitor count",
            "% of total visitors": "% of total visitors", "distance": "Distance from site",
            "Medium Rent": "Medium Rent", "Median Sale Price": "Median Sale Price",
            "final_num_units": "Number of Units"}
    for k, v in cols.items():
        if k not in comp_df.columns.tolist():
            comp_df[k] = None
    comp_df = comp_df.rename(columns=cols)
    comp_df = comp_df[list(cols.values())]
    comp_df['Distance from site'] = comp_df['Distance from site'].apply(lambda x: f"{x} km")
    ws.batch_update([{"range": "E121:K124", "values": comp_df.head(4).values.tolist()}])


In [185]:
def population_chart():
    pop = pd.DataFrame([all_data['data']['population']['15m_catchment']]).T.reset_index(names=['pop']).query(
        "pop.str.startswith('p_')").values.tolist()
    ws.batch_update([{"range": "N18:O35", "values": pop}])


population_chart()

In [190]:
update_category_count()
update_competition()
update_high_streets()
update_projects()

In [21]:
from dashboard import Dashboard

ImportError: cannot import name 'create_report_from_template' from partially initialized module 'dashboard' (most likely due to a circular import) (/home/jyotiraditya/PycharmProjects/SiteReports/dashboard.py)

In [1]:
import pandas as pd

In [3]:
df = pd.DataFrame(columns=['cluster_name', 'distance', 'area', ])

In [4]:
df[['cluster_name', 'distance', 'area', ]]

,cluster_name,distance,area


In [22]:
from typing import Tuple, List, Union
from pydantic import BaseModel


class Cluster(BaseModel):
    cluster_name: Union[str, None]
    locality: Union[str, None]
    polygon_updated: Union[str, None]
    area: Union[float, None]
    lat: Union[float, None]
    lng: Union[float, None]
    distance: Union[float, None]


class Clusters(BaseModel):
    clust: List[Cluster]



In [10]:
df = pd.read_csv("/home/jyotiraditya/PycharmProjects/SiteReports/data/output/1/15m_catchment/high_streets.csv")

In [12]:
df = df.to_dict('records')

In [28]:
df = pd.DataFrame()  #[['cluster_name', 'distance', 'area', ]]
cols = {"cluster_name": "High Streets", "Avg Daily visits": "Avg Daily visits",
        "% of Catchment visitors": "% of Catchment visitors", "distance": "Distance from Site",
        'area': "Total Area"}
for k, v in cols.items():
    if k not in df.columns.tolist():
        df[k] = None
df.rename(columns=cols, inplace=True)
df = df[cols.values()]
df['Total Area'] = df['Total Area'].apply(lambda x: f"{x / 10 ** 6:.2f} sqkm")
df['Distance from Site'] = df['Distance from Site'].apply(lambda x: f"{x} km")

In [29]:
df

,High Streets,Avg Daily visits,% of Catchment visitors,Distance from Site,Total Area


In [27]:
manager.write_dataframe_to_worksheet(df, self.sheet.get_worksheet(self.worksheets['high_streets']))
self.report_worksheet.batch_update([{"range": "E107:I110", "values": df.head(4).values.tolist()}])

TypeError: 'Clusters' object is not subscriptable